# Loading data from preprocessed pickle file

In [1]:
import pickle

# open the pickle file in read-binary mode
with open('..//model-data/preprocessing/train-val-sets/train_val_split.pkl', 'rb') as f:
    data = pickle.load(f)

# storing data
images_train = data['images_train']
images_val = data['images_val']
labels_train = data['labels_train']
labels_val = data['labels_val']

# Creating model

In [7]:
import tensorflow as tf
import keras
from keras import layers, Sequential

model = keras.Sequential([
    layers.Rescaling(scale=1.0/255, input_shape=(256, 256, 3)),
    
    layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    
    layers.Flatten(),
    layers.Dense(39, activation='softmax')
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/preprocessing/tf_data_layer.py:19: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
import tensorflow as tf
import keras
from keras import layers, Sequential

model = keras.Sequential([
    layers.Input(shape=(256, 256, 3)),
    layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'),
    layers.Flatten(),
    layers.Dense(16, activation='softmax'),
    layers.Dense(39, activation='softmax')
])

# Compling the model

In [3]:
# compiling the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Fitting the model

In [4]:
# sliding window model training (to handle batching due to tensorflow gpu mem errors)

for w in range(2):
    # calculating a third of the data size
    third_of_data = len(images_train) // 3
    HR_size = third_of_data
    LR_size = 0
    
    for i in range(3):
        #print(LR_size, HR_size)
        images_train_third = images_train[LR_size:HR_size]
        labels_train_third = labels_train[LR_size:HR_size]
    
        model.fit(images_train_third, labels_train_third, epochs=1)
        
        LR_size += third_of_data;
        HR_size += third_of_data

I0000 00:00:1737827764.478877   25262 service.cc:145] XLA service 0x7815c8005150 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1737827764.478902   25262 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce RTX 4070 Laptop GPU, Compute Capability 8.9
2025-01-25 17:56:04.492222: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-01-25 17:56:04.553003: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8906
I0000 00:00:1737827765.168017   25326 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_450', 544 bytes spill stores, 544 bytes spill loads



  5/440 ━━━━━━━━━━━━━━━━━━━━ 17s 39ms/step - accuracy: 0.1202 - loss: 141.8384 

I0000 00:00:1737827766.816689   25262 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


440/440 ━━━━━━━━━━━━━━━━━━━━ 20s 39ms/step - accuracy: 0.3632 - loss: 35.8232
440/440 ━━━━━━━━━━━━━━━━━━━━ 17s 39ms/step - accuracy: 0.5467 - loss: 2.6793
440/440 ━━━━━━━━━━━━━━━━━━━━ 17s 39ms/step - accuracy: 0.6158 - loss: 1.7074
440/440 ━━━━━━━━━━━━━━━━━━━━ 17s 39ms/step - accuracy: 0.9320 - loss: 0.2590
440/440 ━━━━━━━━━━━━━━━━━━━━ 17s 39ms/step - accuracy: 0.9560 - loss: 0.1578
440/440 ━━━━━━━━━━━━━━━━━━━━ 17s 39ms/step - accuracy: 0.9411 - loss: 0.2120


# Validating the model

In [5]:
import numpy as np
from sklearn.metrics import accuracy_score

# converting one hot encoded labels to class indices
real_labels = np.argmax(labels_val, axis=1)

# predict on the validation set
predictions = model.predict(images_val)

# converting predictions from one hot encoded probabiliti|es to class indices
predicted_labels = np.argmax(predictions, axis=1)

# comparing predictions with the real labels
print("Validation Accuracy:", accuracy_score(real_labels, predicted_labels))

331/331 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step
Validation Accuracy: 0.6474765647192501


# [TESTING]

In [ ]:
import numpy as np

def one_hot_max(arr):
    result = np.zeros_like(arr)
    result[np.argmax(arr)] = 1
    return result

print(one_hot_max(predictions[4]))

In [ ]:
print(labels_val[4])